In [7]:
import cv2
import time
import onnxruntime as ort
import numpy as np

# Chuẩn bị ảnh đầu vào mẫu
img_path = "C:/Users/Trung/Downloads/duc.png"  # ảnh chứa 1 hoặc nhiều khuôn mặt
img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Hàm benchmark thời gian chạy
def benchmark(func, iterations=20):
    times = []
    for _ in range(iterations):
        start = time.time()
        func()
        times.append(time.time() - start)
    return np.mean(times), np.std(times)

# Viola-Jones (Haar Cascade) - chỉ hỗ trợ CPU
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
def detect_haar():
    _ = face_cascade.detectMultiScale(gray, 1.3, 5)

haar_mean, haar_std = benchmark(detect_haar)

# YuNet (ONNX) - hỗ trợ CPU và GPU

def detect_yunet_cpu():
    yunet_cpu = cv2.FaceDetectorYN.create(
        model="D:/Door_Lock_App/models/face_detection_yunet_2023mar.onnx",
        config="",
        input_size=(img.shape[1], img.shape[0]),
        score_threshold=0.9,
        backend_id=cv2.dnn.DNN_BACKEND_DEFAULT,
        target_id=cv2.dnn.DNN_TARGET_CPU
    )
    _ = yunet_cpu.detect(img)

def detect_yunet_gpu():
    yunet_gpu = cv2.FaceDetectorYN.create(
        model="D:/Door_Lock_App/models/face_detection_yunet_2023mar.onnx",
        config="",
        input_size=(img.shape[1], img.shape[0]),
        score_threshold=0.9,
        backend_id=cv2.dnn.DNN_BACKEND_CUDA,
        target_id=cv2.dnn.DNN_TARGET_CUDA
    )
    _ = yunet_gpu.detect(img)


yunet_cpu_mean, yunet_cpu_std = benchmark(detect_yunet_cpu)
# yunet_gpu_mean, yunet_gpu_std = benchmark(detect_yunet_gpu)

# EdgeFace (ONNX) nhận diện khuôn mặt từ ảnh đã cắt
edgeface_sess_cpu = ort.InferenceSession("D:/Door_Lock_App/models/edgeface_s_gamma_05.onnx", providers=["CPUExecutionProvider"])
if 'CUDAExecutionProvider' in ort.get_available_providers():
    edgeface_sess_gpu = ort.InferenceSession("D:/Door_Lock_App/models/edgeface_s_gamma_05.onnx", providers=["CUDAExecutionProvider"])
else:
    edgeface_sess_gpu = None

dummy_face = np.random.rand(1, 3, 112, 112).astype(np.float32)  # ảnh khuôn mặt giả


def recognize_edgeface_cpu():
    _ = edgeface_sess_cpu.run(None, {"input.1": dummy_face})

def recognize_edgeface_gpu():
    if edgeface_sess_gpu:
        _ = edgeface_sess_gpu.run(None, {"input.1": dummy_face})

edge_cpu_mean, edge_cpu_std = benchmark(recognize_edgeface_cpu)
edge_gpu_mean, edge_gpu_std = benchmark(recognize_edgeface_gpu) if edgeface_sess_gpu else (None, None)

# In kết quả
print("⏱️ Benchmark kết quả:")
print(f"Viola-Jones (CPU): {haar_mean:.4f} ± {haar_std:.4f} sec")
print(f"YuNet (CPU):       {yunet_cpu_mean:.4f} ± {yunet_cpu_std:.4f} sec")
# print(f"YuNet (GPU):       {yunet_gpu_mean:.4f} ± {yunet_gpu_std:.4f} sec")
print(f"EdgeFace (CPU):    {edge_cpu_mean:.4f} ± {edge_cpu_std:.4f} sec")
if edge_gpu_mean:
    print(f"EdgeFace (GPU):    {edge_gpu_mean:.4f} ± {edge_gpu_std:.4f} sec")
else:
    print("EdgeFace (GPU):    ❌ GPU không khả dụng")


⏱️ Benchmark kết quả:
Viola-Jones (CPU): 0.0165 ± 0.0044 sec
YuNet (CPU):       0.0271 ± 0.0028 sec
EdgeFace (CPU):    0.0084 ± 0.0076 sec
EdgeFace (GPU):    0.0086 ± 0.0089 sec


In [8]:
import onnxruntime as ort
import numpy as np
import time

# Thay đổi cho phù hợp với input thực tế của bạn
input_shape = (1, 3, 128, 128)  # Ví dụ mô hình nhận diện khuôn mặt EdgeFace
dummy_input = np.random.rand(*input_shape).astype(np.float32)

def benchmark_onnx(session, input_name, input_tensor, iterations=50):
    times = []
    for _ in range(iterations):
        start = time.time()
        session.run(None, {input_name: input_tensor})
        times.append(time.time() - start)
    return np.mean(times), np.std(times)


# Tên input biết trước
input_name = "input"  # thay bằng tên đúng của bạn

# Load ONNX model
model_path = "D:/Door_Lock_App/models/AntiSpoofing_cls2_bbox2_sz128_128_best.onnx"  # thay bằng đường dẫn của bạn

# Session CPU
sess_cpu = ort.InferenceSession(model_path, providers=["CPUExecutionProvider"])

# Session GPU (nếu hỗ trợ)
sess_gpu = None
if "CUDAExecutionProvider" in ort.get_available_providers():
    sess_gpu = ort.InferenceSession(model_path, providers=["CUDAExecutionProvider"])

cpu_mean, cpu_std = benchmark_onnx(sess_cpu, input_name, dummy_input)
print(f"🧠 CPU Inference: {cpu_mean:.4f} ± {cpu_std:.4f} seconds")

if sess_gpu:
    gpu_mean, gpu_std = benchmark_onnx(sess_gpu, input_name, dummy_input)
    print(f"🚀 GPU Inference: {gpu_mean:.4f} ± {gpu_std:.4f} seconds")
else:
    print("❌ GPU không khả dụng hoặc onnxruntime-gpu chưa được cài")



🧠 CPU Inference: 0.0056 ± 0.0053 seconds
🚀 GPU Inference: 0.0056 ± 0.0057 seconds
